## Homework: Search Evaluation

In this homework, we will evaluate the results of vector
search.

> It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Required libraries

We will use minsearch and Qdrant. Make sure you have the most up-to-date versions:

```bash
pip install -U minsearch qdrant_client
``` 

minsearch should be at least 0.0.4.



## Evaluation data

For this homework, we will use the same dataset we generated
in the videos.

Let's get them:

```python
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')
```

Here, `documents` contains the documents from the FAQ database
with unique IDs, and `ground_truth` contains generated
question-answer pairs. 

Also, we will need the code for evaluating retrieval:

```python
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }
```



## Q1. Minsearch text

Now let's evaluate our usual minsearch approach, indexing documents with:
```python
text_fields=["question", "section", "text"],
keyword_fields=["course", "id"]
```
but tweak the parameters for search. Let's use the following boosting params:

```python
boost = {'question': 1.5, 'section': 0.1}
```

What's the hitrate for this approach?

* 0.64
* 0.74
* 0.84
* 0.94

In [ ]:
!pip install -U minsearch qdrant_client ipwidget jupyter

In [ ]:
# Step 1: Install and import necessary libraries
import json
import requests
import pandas as pd
from minsearch import AppendableIndex
from tqdm.auto import tqdm

# Step 2: Load documents from Alexey's repo
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

# Step 3: Flatten the nested JSON into a flat list of documents
documents = []

for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name  # Add course name to each doc
        documents.append(doc)

# Step 4: Load the ground truth data (for evaluation)
url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
gt_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(gt_url)
ground_truth = df_ground_truth.to_dict(orient='records')

# Step 5: Build the MinSearch index
index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]  # include "id" for correct match
)

index.fit(documents)

# Step 6: Define the search function with filtering and boosting
def search(q):
    boost = {'question': 1.5, 'section': 0.1}
    return index.search(
        query=q['question'],                           # user's question
        filter_dict={'course': q['course']},           # filter by course
        boost_dict=boost,                              # field boosting
        num_results=5,
        output_ids=True                                # include document IDs in result
    )

# Step 7: Define evaluation metrics
def hit_rate(relevance_total):
    cnt = 0
    for line in relevance_total:
        if True in line:
            cnt += 1
    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0
    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score += 1 / (rank + 1)
                break
    return total_score / len(relevance_total)

# Step 8: Define main evaluation loop
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)  # run search on question
        relevance = [d['id'] == doc_id for d in results]  # check if result is correct
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

# Step 9: Run evaluation
results = evaluate(ground_truth, search)

# Step 10: Print the results
print(results)


  0%|          | 0/4627 [00:00<?, ?it/s]

KeyError: 'id'

In [4]:
import minsearch
minsearch.__version__

'0.0.4'